In [1]:
import pandas as pd
import numpy as np
import pickle
import psycopg2
from sqlalchemy import Table, Column, Integer, String, Float, Numeric
from sqlalchemy import create_engine, MetaData
import os
from io import StringIO

In [2]:
df_mech = pickle.load(open(r'C:\Users\tdkla\PycharmProjects\boardgame_recommender_system\similarity_scores_mech_user.pkl', 'rb'))

In [3]:
df_all = pickle.load(open(r'C:\Users\tdkla\PycharmProjects\boardgame_recommender_system\similarity_scores_all_features.pkl', 'rb'))

In [4]:
df_theme = pickle.load(open(r'C:\Users\tdkla\PycharmProjects\boardgame_recommender_system\similarity_scores_theme_user.pkl', 'rb'))

In [5]:
df_games = pickle.load(open(r'C:\Users\tdkla\PycharmProjects\boardgame_recommender_system\games.pkl', 'rb'))

In [6]:
# Check for duplicate game names
df_dup = df_games[df_games.duplicated(subset='Name')]

In [ ]:
# If games are duplicates, we will append the bggid to the end,
# as there are multiple games with identical names published within the same year (e.g. Loch Ness (2010))
if not df_dup.empty:
    print("Duplicate game names found:")
    
    # Append year to duplicate game names
    for game_name in df_dup['Name']:
        bggid = df_games.loc[df_games['Name'] == game_name, 'BGGId']
        print(bggid.values)
        df_games.loc[df_games['Name'] == game_name, 'Name'] = game_name + ' (' + bggid.astype(str) + ')'
    
    # Check for duplicates again after appending year
    df_dup_after_append = df_games[df_games.duplicated(subset='Name')]
    if df_dup_after_append.empty:
        print("Duplicates resolved by appending year")
    else:
        print("Duplicates remain even after appending BGGID:")
        print(df_dup_after_append)
else:
    print("No duplicate game names found")


In [8]:
# check if empty
df_dup_after_append

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens


In [9]:
# Update df columns with game names
df_all = pd.DataFrame(df_all, columns=df_games['Name'])
df_mech = pd.DataFrame(df_mech, columns=df_games['Name'])
df_theme = pd.DataFrame(df_theme, columns=df_games['Name'])

In [10]:
df_mech.head()

Name,Die Macher,Dragonmaster,Samurai (3),Tal der Könige,Acquire,Mare Mediterraneum,Cathedral,Lords of Creation,El Caballero,Elfenland,...,System Gateway (fan expansion for Android: Netrunner),Voll verplant,Mille Fiori,7 Wonders: Architects,Azul: Queen's Garden,Salvage,Blitzkrieg!: World War Two in 20 Minutes (347521),Rock Paper Scissors: Deluxe Edition,Splitter,Captains' War
0,1.000000,0.006483,0.309278,0.354869,0.161819,0.347149,0.012717,0.009779,0.262480,0.195305,...,0.120268,0.001887,0.002297,0.002694,0.002204,0.242503,0.241079,0.170181,0.242753,0.170084
1,0.006483,1.000000,0.007463,0.006273,0.010425,0.003876,0.008852,0.004256,0.007409,0.008627,...,0.000000,0.000000,0.001381,0.000758,0.000386,0.636396,0.000491,0.000000,0.000803,0.000000
2,0.309278,0.007463,1.000000,0.378431,0.316664,0.004233,0.200978,0.007058,0.540723,0.216335,...,0.130664,0.002241,0.002354,0.216773,0.371242,0.002821,0.261445,0.000539,0.002536,0.000000
3,0.354869,0.006273,0.378431,1.000000,0.011692,0.019233,0.011942,0.018050,0.339089,0.014644,...,0.159099,0.002829,0.004087,0.261656,0.227451,0.320575,0.159099,0.000000,0.003418,0.000000
4,0.161819,0.010425,0.316664,0.011692,1.000000,0.005565,0.189818,0.007829,0.260018,0.200576,...,0.120960,0.001917,0.001543,0.003741,0.173360,0.002274,0.121776,0.000791,0.001624,0.000000


In [11]:
# Create df to be SQL table for games information
df_info =pd.DataFrame()
df_info[['Name', 'ImagePath', 'YearPublished', 'MfgPlaytime',
         'MinPlayers', 'MaxPlayers', 'AvgRating']] = df_games[['Name', 'ImagePath','YearPublished',
                                                               'MfgPlaytime', 'MinPlayers', 'MaxPlayers', 'AvgRating']]

df_info.head()

,Name,ImagePath,YearPublished,MfgPlaytime,MinPlayers,MaxPlayers,AvgRating
0,Die Macher,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,1986,240,3,5,7.61428
1,Dragonmaster,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,1981,30,3,4,6.64537
2,Samurai (3),https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,1998,60,2,4,7.45601
3,Tal der Könige,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,1992,60,2,4,6.60006
4,Acquire,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...,1964,90,2,6,7.33861


In [34]:
# Establish a connection to the PostgreSQL database
connection_params = {
    'host': dbhost,
    'port': dbport,
    'dbname': dbname,
    'user': dbuser,
    'password': dbpass,
    'sslmode': dbssl
}
    
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**connection_params)
# Create a cursor object
cursor = conn.cursor()

# Enable autocommit mode
conn.autocommit = True

In [35]:
# create mapping to remove special characters not allowed in SQL table names
character_map = {' ':'_', '!': '_', '*': '_', '^': '_', '%': '_', '$': '_', '&': 'and',
                 ':': '_', '#': '_', "'": "_", '(': '_', ')': '_', '/': '_',
                 '.': '_', ',': '_', '-': '_', '?':'_', '`':'_', '+':'_', '"':"_", '[':'_',
                ']':'_', '=':'_', '@':'_'}

# Iterate over each column in the dataframe
for column_name in df_all.columns:
    # Generate a temporary table name for the current column
    table_name = f"table_{column_name.lower()}"
    
    for char, replacement in character_map.items():
        table_name = table_name.replace(char, replacement)
        

    try:

        # Drop the table if it already exists
        drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
        cursor.execute(drop_table_query)

        # Create the temporary table
        create_table_query = f"CREATE TABLE {table_name} (id FLOAT PRIMARY KEY, combined FLOAT, mech FLOAT, theme FLOAT)"
        cursor.execute(create_table_query)

        # Generate the list of index values (increasing integers)
        index_values = list(range(len(df_games)))

         # Check if the encoded column name exists in df_similarity
        if column_name in df_all.columns:

            #print(column_name)
            # Convert the data to a string and create a StringIO object
            values_all = df_all[column_name].tolist()
            values_mech = df_mech[column_name].tolist()
            values_theme = df_theme[column_name].tolist()

            # Create a StringIO object to hold the data
            data_io = StringIO()

            # Write the index and value pairs to the StringIO object
            for index, value_all, value_mech, value_theme in zip(index_values, values_all, values_mech, values_theme):
                data_io.write(f"{index},{value_all},{value_mech},{value_theme}\n")

            # Reset the buffer position
            data_io.seek(0)

            # Copy the data from the StringIO object to the temporary table using COPY FROM
            copy_query = f"COPY {table_name} FROM STDIN WITH CSV"
            cursor.copy_expert(copy_query, data_io)

            # Commit the changes for the current column
            conn.commit()
        else:
            print(f"Error: Column '{column_name}' not found in df_similarity")

    except Exception as e:
        #Handle the exception and perform a rollback
        print(f"Error occurred: {str(e)}")
        conn.rollback()
        
# Close the cursor and the database connection
cursor.close()
conn.close()

In [23]:
# Single game insert
# create mapping to remove special characters not allowed in SQL table names
character_map = {' ':'_', '!': '_', '*': '_', '^': '_', '%': '_', '$': '_', '&': 'and',
                 ':': '_', '#': '_', "'": '_', '(': '_', ')': '_', '/': '_',
                 '.': '_', ',': '_', '-': '_', '?':'_', '`':'_', '+':'_', '"':'_', '[':'_',
                ']':'_', '=':'_', '@':'_'}

# Iterate over each column in the dataframe
for column_name in ['News@11']:
    # Generate a temporary table name for the current column
    table_name = f"table_{column_name.lower()}"
    
    for char, replacement in character_map.items():
        table_name = table_name.replace(char, replacement)
        
    try:

        # Drop the table if it already exists
        drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
        cursor.execute(drop_table_query)

        # Create the temporary table
        create_table_query = f"CREATE TABLE {table_name} (id FLOAT PRIMARY KEY, combined FLOAT, mech FLOAT, theme FLOAT)"
        cursor.execute(create_table_query)

        # Generate the list of index values (increasing integers)
        index_values = list(range(len(df_games)))

         # Check if the encoded column name exists in df_similarity
        if column_name in df_all.columns:

            #print(column_name)
            # Convert the data to a string and create a StringIO object
            values_all = df_all[column_name].tolist()
            values_mech = df_mech[column_name].tolist()
            values_theme = df_theme[column_name].tolist()

            # Create a StringIO object to hold the data
            data_io = StringIO()

            # Write the index and value pairs to the StringIO object
            for index, value_all, value_mech, value_theme in zip(index_values, values_all, values_mech, values_theme):
                data_io.write(f"{index},{value_all},{value_mech},{value_theme}\n")

            # Reset the buffer position
            data_io.seek(0)

            # Copy the data from the StringIO object to the temporary table using COPY FROM
            copy_query = f"COPY {table_name} FROM STDIN WITH CSV"
            cursor.copy_expert(copy_query, data_io)

            # Commit the changes for the current column
            conn.commit()
        else:
            print(f"Error: Column '{column_name}' not found in df_similarity")

    except Exception as e:
        #Handle the exception and perform a rollback
        print(f"Error occurred: {str(e)}")
        conn.rollback()
        
# Close the cursor and the database connection
cursor.close()
conn.close()

Error occurred: name 'df_all' is not defined


In [21]:
# create SQL table for df_info

# create mapping to remove special characters not allowed in SQL table names
character_map = {' ':'_', '!': '_', '*': '_', '^': '_', '%': '_', '$': '_', '&': 'and',
                 ':': '_', '#': '_', "'": '_', '(': '_', ')': '_', '/': '_',
                 '.': '_', ',': '_', '-': '_', '?':'_', '`':'_', '+':'_', '"':'_', '[':'_',
                ']':'_', '=':'_', '@':'_'}

# Generate a temporary table name for the current column
table_name = f"table_game_info"


try:

    # Drop the table if it already exists
    drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(drop_table_query)

    # Create the temporary table
    create_table_query = f"CREATE TABLE {table_name} (id FLOAT PRIMARY KEY, name VARCHAR, image TEXT, year_pub VARCHAR, " \
                         f"play_time FLOAT, min_players INT, max_players INT, avg_rating FLOAT)"

    cursor.execute(create_table_query)

    # Generate the list of index values (increasing integers)
    index_values = list(range(len(df_info)))

    #print(column_name)
    # Convert the data to a string and create a StringIO object
    values_name = df_info['Name'].tolist()
    values_image = df_info['ImagePath'].tolist()
    values_year = df_info['YearPublished'].tolist()
    values_playtime = df_info['MfgPlaytime'].tolist()
    values_minplay = df_info['MinPlayers'].tolist()
    values_maxplay = df_info['MaxPlayers'].tolist()
    values_rating = df_info['AvgRating'].tolist()

    # Create a StringIO object to hold the data
    data_io = StringIO()

    # Write the index and value pairs to the StringIO object
    for index, value_name, value_image, value_year, value_play, value_min,\
    value_max, value_rate in zip(index_values, values_name, values_image,\
                                 values_year, values_playtime, values_minplay, values_maxplay, values_rating):

        data_io.write(f"{index},\"{value_name}\",\"{value_image}\",{value_year},{value_play},{value_min},{value_max},{value_rate}\n")

    # Reset the buffer position
    data_io.seek(0)

    # Copy the data from the StringIO object to the temporary table using COPY FROM
    copy_query = f"COPY {table_name} FROM STDIN WITH CSV"
    cursor.copy_expert(copy_query, data_io)

    # Commit the changes for the current column
    conn.commit()

except Exception as e:
    #Handle the exception and perform a rollback
    print(f"Error occurred: {str(e)}")
    conn.rollback()
        
# Close the cursor and the database connection
cursor.close()
conn.close()

In [36]:
# Check if the commit was successful
if conn.status == psycopg2.extensions.STATUS_IN_TRANSACTION:
    print("Commit failed")
else:
    print("Commit successful")

Commit successful
